In [1]:
#importing libraries
import pandas as pd
import tqdm as tqdm

In [2]:
#reading in the data
stats = pd.read_csv("all_stats.csv")

In [3]:
#
stats

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,WS/48,OBPM,DBPM,BPM,VORP,PTS_T,AST_R,STL_R,BLK_R,3P_R
0,A.C. Green,SF,32,PHO,82,36,25.8,2.6,5.4,0.484,...,0.100,-0.4,-0.9,-1.3,0.4,0.889320,0.466739,0.723462,0.697826,0.402634
1,A.C. Green,PF,37,MIA,82,1,17.2,1.8,4.0,0.444,...,0.110,-2.2,0.0,-2.2,-0.1,0.576052,0.274186,0.596349,0.228025,0.000000
2,A.C. Green,PF,33,DAL,83,73,30.0,2.8,5.8,0.483,...,0.093,-1.0,-0.4,-1.4,0.4,0.896924,0.439243,1.136598,0.472416,0.000000
3,A.C. Green,PF,34,DAL,82,68,32.3,3.0,6.5,0.453,...,0.095,-1.1,-0.3,-1.4,0.4,0.918463,0.824362,1.394536,0.708445,0.000000
4,A.C. Green,PF,35,DAL,50,35,18.5,2.2,5.1,0.422,...,0.074,-1.1,-1.1,-2.2,0.0,0.667203,0.302863,0.888889,0.498584,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16028,Željko Rebrača,C,30,DET,30,12,16.3,2.7,4.8,0.552,...,0.133,-2.8,0.2,-2.6,-0.1,0.840864,0.168261,0.297429,1.383621,0.000000
16029,Željko Rebrača,C,29,DET,74,4,15.9,2.6,5.1,0.505,...,0.134,-2.9,0.6,-2.3,-0.1,0.864440,0.271504,0.598029,2.231237,0.000000
16030,Željko Rebrača,C,32,LAC,58,2,16.0,2.3,4.0,0.568,...,0.122,-1.7,0.4,-1.3,0.2,0.717099,0.225462,0.309436,1.693431,0.000000
16031,Željko Rebrača,C,33,LAC,29,2,14.2,1.8,3.3,0.542,...,0.086,-3.7,0.9,-2.8,-0.1,0.588802,0.177611,0.331404,1.764447,0.000000


In [4]:
#fill in missing values with 0
stats = stats.fillna(0)

In [5]:
stats.columns

Index(['Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year',
       'Pts Won', 'Pts Max', 'Share', 'Team', 'W', 'L', 'W/L%', 'GB', 'PS/G',
       'PA/G', 'SRS', 'Rk', 'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%',
       'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'OWS', 'DWS', 'WS',
       'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP', 'PTS_T', 'AST_R', 'STL_R',
       'BLK_R', '3P_R'],
      dtype='object')

In [6]:
#setting up the predictors
predictors = ['Age', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year',
       'W', 'L', 'W/L%', 'GB', 'PS/G',
       'PA/G', 'SRS', 'Rk', 'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%',
       'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'OWS', 'DWS', 'WS',
       'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP']

In [7]:
#split data into training and testing sets
train = stats[stats["Year"] < 2025]
test = stats[stats["Year"] == 2025]

In [8]:
#function to find the average precision
def find_ap(combination):
    actual = combination.sort_values("Share", ascending=False).head(5)
    predicted = combination.sort_values("predictions", ascending=False)
    ps=[]
    found=0
    seen=1
    for index, row in predicted.iterrows():
        if row["Player"] in actual["Player"].values:
            found +=1
            ps.append(found/seen)
        seen +=1
    return sum(ps)/len(ps)



In [9]:
#function to add ranks to the combination dataframe
def add_ranks(combination):
    # Add actual ranks
    combination = combination.sort_values("Share", ascending=False)
    combination["Rk"] = list(range(1, combination.shape[0] + 1))
    combination['Rk'] = combination['Rk'].apply(lambda x: x if x <= 10 else 11)
    # Add predicted ranks
    combination = combination.sort_values("predictions", ascending=False)
    combination["Predicted_Rk"] = list(range(1, combination.shape[0] + 1))
    combination['Predicted_Rk'] = combination['Predicted_Rk'].apply(lambda x: x if x <= 10 else 11)
    combination["Diff"] = combination["Rk"] - combination["Predicted_Rk"]
    return combination


In [10]:
#Backtesting the model
def backtest(stats, model, year, predictors):
    aps = []
    all_predictions = []
    for year in tqdm.tqdm(years[5:]):
        train = stats[stats["Year"] < year]
        test = stats[stats["Year"] == year]

        model.fit(train[predictors], train["Share"])
        predictions = model.predict((test[predictors]))
        predictions = pd.DataFrame(predictions, columns=["predictions"], index=test.index)
        combination = pd.concat([test[["Player", "Share"]], predictions], axis=1)
        combination = add_ranks(combination)
        all_predictions.append(combination)
        aps.append(find_ap(combination))
    return sum(aps)/len(aps), aps, pd.concat(all_predictions)
    

In [11]:
#adding ratios to the stats dataframe
# stat_ratios = stats[["PTS", "AST", "STL", "BLK", "3P", "Year"]].groupby("Year").apply(lambda x: x/x.mean())
# stat_ratios = stat_ratios.reset_index(level=0, drop=True)
# stats[["PTS_T", "AST_R", "STL_R", "BLK_R", "3P_R"]] = stat_ratios[["PTS", "AST", "STL", "BLK", "3P"]]

In [12]:
# stats.to_csv("all_stats.csv", index=False)

In [13]:
#adding stats ratios to the predictors
# predictors += ["PTS_T", "AST_R", "STL_R", "BLK_R", "3P_R"]

In [14]:
years = list(range(1991, 2026))

In [32]:
from sklearn.ensemble import HistGradientBoostingRegressor

hgb = HistGradientBoostingRegressor(random_state=1, max_iter=200, learning_rate=0.01)

hgb.fit(train[predictors], train["Share"])

predictions = hgb.predict(test[predictors])
predictions = pd.DataFrame(predictions, columns=["predictions"], index=test.index)
combination = pd.concat([test[["Player", "Share"]], predictions], axis=1)
ap = find_ap(combination)
add_ranks(combination), ap


(                        Player  Share  predictions  Rk  Predicted_Rk  Diff
 13676  Shai Gilgeous-Alexander  0.913     0.743835   1             1     0
 11649             Nikola Jokić  0.787     0.534912   2             2     0
 5675     Giannis Antetokounmpo  0.470     0.247551   3             3     0
 7286              Jayson Tatum  0.311     0.049419   4             4     0
 821            Anthony Edwards  0.012     0.045381   7             5     2
 ...                        ...    ...          ...  ..           ...   ...
 15972          Zion Williamson  0.000    -0.000085  11            11     0
 7028             Jarrett Allen  0.000    -0.000165  11            11     0
 8600        Karl-Anthony Towns  0.000    -0.000899  11            11     0
 15189                Ty Jerome  0.000    -0.001007  11            11     0
 6373               Ivica Zubac  0.000    -0.002970  11            11     0
 
 [488 rows x 6 columns],
 0.82)

In [33]:
#using the HistGradientBoostingRegressor


mean_ap, aps, all_predictions = backtest(stats, hgb, years[5:], predictors)


100%|██████████| 30/30 [02:58<00:00,  5.95s/it]


In [34]:
mean_ap

0.8026893943240877

In [46]:
#importing the models
from sklearn.ensemble import HistGradientBoostingRegressor, GradientBoostingRegressor, VotingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
lgb_model = LGBMRegressor(n_estimators=100, random_state=1)
hgb_model = HistGradientBoostingRegressor(random_state=1, max_iter=100)
xg_model = XGBRegressor(random_state=1, n_estimators=100, learning_rate=0.01)
#fitting the model
voting_model = VotingRegressor(estimators=[('lgb', lgb_model), ('hgb', hgb_model), ('xg', xg_model)])
voting_model.fit(train[predictors], train["Share"])
#making predictions for 2025
predictions = voting_model.predict(test[predictors])
predictions = pd.DataFrame(predictions, columns=["predictions"], index=test.index)









[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.012739 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 9027
[LightGBM] [Info] Number of data points in the train set: 15545, number of used features: 55
[LightGBM] [Info] Start training from score 0.005686


In [47]:
combination = pd.concat([test[["Player", "Share"]], predictions], axis=1)
ap = find_ap(combination)
add_ranks(combination), ap





(                        Player  Share  predictions  Rk  Predicted_Rk  Diff
 13676  Shai Gilgeous-Alexander  0.913     0.794371   1             1     0
 11649             Nikola Jokić  0.787     0.571223   2             2     0
 5675     Giannis Antetokounmpo  0.470     0.304992   3             3     0
 7286              Jayson Tatum  0.311     0.084043   4             4     0
 14962               Trae Young  0.000     0.070645  11             5     6
 ...                        ...    ...          ...  ..           ...   ...
 15972          Zion Williamson  0.000    -0.003411  11            11     0
 14034            Stephen Curry  0.002    -0.003414   9            11    -2
 6622             Jalen Brunson  0.001    -0.003930  11            11     0
 8941              Kevin Durant  0.000    -0.004852  11            11     0
 7824               Joel Embiid  0.000    -0.006753  11            11     0
 
 [488 rows x 6 columns],
 0.8277777777777778)

In [51]:

#backtest
mean_ap, aps, all_predictions = backtest(stats, voting_model, years[5:], predictors)

  0%|          | 0/30 [00:00<?, ?it/s]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002843 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7286
[LightGBM] [Info] Number of data points in the train set: 1969, number of used features: 55
[LightGBM] [Info] Start training from score 0.006601
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


  3%|▎         | 1/30 [00:02<01:14,  2.56s/it]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.019434 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7566
[LightGBM] [Info] Number of data points in the train set: 2397, number of used features: 55
[LightGBM] [Info] Start training from score 0.006509
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


  7%|▋         | 2/30 [00:06<01:27,  3.11s/it]

...

 93%|█████████▎| 28/30 [01:51<00:14,  7.15s/it]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007865 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 9013
[LightGBM] [Info] Number of data points in the train set: 15051, number of used features: 55
[LightGBM] [Info] Start training from score 0.005700


 97%|█████████▋| 29/30 [01:55<00:06,  6.12s/it]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.009096 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 9027
[LightGBM] [Info] Number of data points in the train set: 15545, number of used features: 55
[LightGBM] [Info] Start training from score 0.005686


100%|██████████| 30/30 [01:59<00:00,  3.98s/it]


In [52]:
mean_ap

0.8127250827141989

In [212]:
# See if you're getting better or worse over time
yearly_performance = pd.DataFrame({'Year': years[5:], 'AP': aps})
print(yearly_performance.sort_values('Year').tail(120))

    Year        AP
0   1996  0.701429
1   1997  0.712195
2   1998  0.810000
3   1999  0.386465
4   2000  0.844444
5   2001  0.826190
6   2002  0.790303
7   2003  1.000000
8   2004  0.607075
9   2005  0.420833
10  2006  0.615775
11  2007  0.780952
12  2008  0.925000
13  2009  0.966667
14  2010  0.925000
15  2011  0.794444
16  2012  0.516667
17  2013  0.804762
18  2014  0.683333
19  2015  0.926667
20  2016  0.942857
21  2017  0.850909
22  2018  0.926667
23  2019  0.780952
24  2020  0.925000
25  2021  0.616206
26  2022  0.862500
27  2023  0.826190
28  2024  0.942857
29  2025  0.843478


In [50]:
# Assuming you have existing results
yearly_performance = pd.DataFrame({'Year': years[5:], 'AP': aps})

# Filter to recent years
recent_performance = yearly_performance[yearly_performance['Year'] >= 2019]
recent_ap = recent_performance['AP'].mean()

print(f"AP for years >= 2019: {recent_ap:.4f}")
print(f"Number of years: {len(recent_performance)}")

AP for years >= 2019: 0.8965
Number of years: 7


In [166]:
combination.sort_values("Share", ascending=False).head(10)

,Player,Share,predictions
11645,Nikola Jokić,0.935,0.255345
13672,Shai Gilgeous-Alexander,0.646,0.154796
9945,Luka Dončić,0.572,0.209183
5666,Giannis Antetokounmpo,0.194,0.230050
6626,Jalen Brunson,0.143,0.117840
7285,Jayson Tatum,0.087,0.103313
823,Anthony Edwards,0.018,0.072423
4394,Domantas Sabonis,0.003,0.142944
8939,Kevin Durant,0.001,0.090558
144,Admiral Schofield,0.000,0.004341
